# IML HW3 

For the first question, generating a 50000 * 50000 matrix was effectively impossible with the level of compute available to me. Thereby, I could notice significant difference in speeds of the two PCA functions before going up to this size. Hence, I stopped at a function of size 1000 (samples) * 100 (parameters). 



```cpp
// due to issues setting up eigen path, run with:
// g++ PCA.cpp -o test -I /usr/include/eigen3
#include <chrono>
#include <iostream>
#include <Eigen/Dense>
#include <vector>
#include <assert.h>
#include <fstream>
 
using Eigen::MatrixXd;


using namespace Eigen;

template<typename M>
M load_csv (const std::string & path) {
    std::ifstream indata;
    indata.open(path);
    std::string line;
    std::vector<double> values;
    uint rows = 0;
    while (std::getline(indata, line)) {
        std::stringstream lineStream(line);
        std::string cell;
        while (std::getline(lineStream, cell, ',')) {
            values.push_back(std::stod(cell));
        }
        ++rows;
    }
    return Map<const Matrix<typename M::Scalar, M::RowsAtCompileTime, M::ColsAtCompileTime, RowMajor>>(values.data(), rows, values.size()/rows);
}


Eigen::VectorXd PCA(MatrixXd m){
  // time measurement


  // Generate Covariance matrix 
  Eigen::MatrixXd centered = m.rowwise() - m.colwise().mean(); // subtract column means from each row
  Eigen::MatrixXd Cov = (1.0/(m.rows()-1)) * centered.adjoint() * centered; // calculate covariance matrix
  assert(Cov.rows() == Cov.cols() && "Covariance matrix is not square"); // this is a check
  // find eigenvalues 
  Eigen::EigenSolver<Eigen::MatrixXd> solver(Cov);
  Eigen::VectorXd eigenvalues = solver.eigenvalues().real();
  // since the covariance matrix is always syymetric, are eigenvalues are never complex
  // find largest eigenvalue
  double largest_eigenvalue = eigenvalues.maxCoeff();
  // find eigenvector corresponding to largest eigenvalue
  Eigen::MatrixXcd eigenvectors = solver.eigenvectors(); // find all eigenvectors
  int largest_eigenvalue_index;
  for (int i=0; i<eigenvalues.size(); i++) {
    if (eigenvalues[i] == largest_eigenvalue) { // find the corresponding eigenvector's index
      largest_eigenvalue_index = i;
      break;
    }
  }
  Eigen::VectorXd principal_component = eigenvectors.col(largest_eigenvalue_index).real(); 
  return principal_component;
}

int main()
{
  using std::chrono::high_resolution_clock;
  using std::chrono::duration_cast;
  using std::chrono::duration;
  using std::chrono::milliseconds;

  MatrixXd m = load_csv<MatrixXd>("massivedataset.csv");

  auto t1 = high_resolution_clock::now();
  Eigen::VectorXd output = PCA(m);
  auto t2 = high_resolution_clock::now();
  

  /* Getting number of milliseconds as an integer. */
  auto ms_int = duration_cast<milliseconds>(t2 - t1);

  /* Getting number of milliseconds as a double. */
  duration<double, std::milli> ms_double = t2 - t1;
  std::cout << "time taken is\n";
  // std::cout << ms_int.count() << "ms\n";
  std::cout << ms_double.count() << "ms\n";
  std::cout << "answer is \n";
  std::cout  << output  << std::endl;
}
```

In [1]:
from sklearn.decomposition import PCA
import time
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

data = pd.read_csv(r"C:\Users\hrshv\Documents\CS-1390-Intro-to-Machine-Learning\massivedataset.csv")
# pca = PCA()
# a = pca.fit(data)
# type(a)

 # apply PCA to the input features X
st = time.time()
pca = PCA(n_components=1)
output = pca.fit_transform(data.T)
et = time.time()

elapsed_time = et - st
print('Execution time:', elapsed_time * 1000, 'ms')

Execution time: 28.00154685974121 ms


The sklearn function takes around 100 ms in most cases.

My function takes around 1000 ms, 

so the sklearn function is faster

I am not sure but I think this could be, to a large extent, because operations with the pandas dataframe are faster than with an 'eigen' matrix

In [2]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift


In [3]:
from sklearn.datasets import fetch_openml
import numpy as np

X,y = fetch_openml(
    "mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas"
)

data = pd.DataFrame(X)
target = pd.DataFrame(y)

In [24]:
sample_size = 30000
random_indices = np.random.choice(data.index, size=sample_size, replace=False)
sample_data = data.loc[random_indices]
sample_target = target.loc[random_indices]

In [25]:
from sklearn import metrics
# I did not write the function below. I found an efficient function here:
# https://stackoverflow.com/questions/34047540/python-clustering-purity-metric
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=1)) / np.sum(contingency_matrix) 

In [26]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for GridSearchCV
param_grid = {
    'max_iter': [100, 200, 300],
    'tol': [0.0001, 0.001, 0.01],
}

n_clusters = 10

# Create KMeans estimator
kmeans = KMeans(init='k-means++', n_init=n_clusters, verbose=0, 
                random_state=None, copy_x=True, algorithm='lloyd')

# Create GridSearchCV object
grid_search = GridSearchCV(kmeans, param_grid=param_grid, cv=5)

# Fit GridSearchCV object to the data
grid_search.fit(sample_data)

print("Best parameters: ", grid_search.best_params_)

Best parameters:  {'max_iter': 100, 'tol': 0.0001}


In [31]:
n_clusters = 10

# Create a KMeans instance with 10 clusters
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=n_clusters, 
                max_iter=100, tol=0.0001, verbose=0, random_state=None, 
                copy_x=True, algorithm='lloyd')

# Fit the KMeans model to the sample_data DataFrame
kmeans.fit(sample_data)

# Get the cluster labels for each datapoint in the sample_data DataFrame
kmeans_pred = kmeans.predict(sample_data)
purity_score(sample_target, kmeans_pred)

0.5879666666666666

In [28]:
# dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean', 
#                 metric_params=None, algorithm='auto', leaf_size=30, 
#                 p=None, n_jobs=None)

# dbscan.fit(sample_data)

# dbscan_pred = dbscan.fit_predict(sample_data)

# purity_score(sample_target, dbscan_pred)

I am not able to fix the prediction with dbscan, even after changing the different hyperparameters a lot and trying with GridsearchCV

From what I see here:

https://crunchingthedata.com/when-to-use-dbscan/

the algorithm the algorithm is not suited to solving problems with this dataset

In [29]:
# meanshift = MeanShift(bandwidth=None, seeds=None, bin_seeding=False, 
#                       min_bin_freq=1, cluster_all=True, n_jobs=None, max_iter=100)

# meanshift.fit(sample_data)

# meanshift_pred = meanshift.fit_predict(sample_data)

# purity_score(sample_target, meanshift_pred)

I tried a few things with meanshift, but in this case the main issue is that it is not running fast enough with a significant number of samples. Testing different hyperparameters with >= 1000 samples was effectively impossible

In [30]:
# unique_vals = np.unique(y)

# # Use count_nonzero() to count the occurrence of each unique value in the array
# for val in unique_vals:
#     count = np.count_nonzero(y == val)
#     print(f"{val}: {count}")